In [2]:
!pwd

/home/mchistya/code/eangelid/trampact/notebooks


In [3]:
!pip install lambert

In [147]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from lambert import Lambert93, convertToWGS84Deg

#from iris import get_iris_type
# add iris type to data on T1

# first dataset T1

In [107]:
data = pd.read_csv('../raw_data/BPE_T1.csv')


## Dataset properties

Note sur Lambert
L'expression des coordonnées dans ce système est tridimensionnelle sous forme de longitudes, latitudes et hauteurs ellipsoïdales. 

In [108]:
data.head(5)

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE
0,93,6,6088,06088_0801,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport
1,93,6,6088,06088_1005,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé
2,93,6,6088,06088_1005,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme
3,93,6,6088,06088_3801,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a
4,93,6,6088,06088_3801,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a


In [109]:
pd.unique(data['DEPCOM'])

array([6088, 6159])

In [7]:
data.shape

(11857, 11)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   REG         11857 non-null  int64  
 1   DEP         11857 non-null  int64  
 2   DEPCOM      11857 non-null  int64  
 3   DCIRIS      11857 non-null  object 
 4   AN          11857 non-null  int64  
 5   TYPEQU      11857 non-null  object 
 6   LAMBERT_X   11857 non-null  float64
 7   LAMBERT_Y   11857 non-null  float64
 8   QUALITE_XY  11857 non-null  object 
 9   LAB         11857 non-null  object 
 10  CLASSE      11857 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 1019.1+ KB


In [9]:
data.describe()

,REG,DEP,DEPCOM,AN,LAMBERT_X,LAMBERT_Y
count,11857.0,11857.0,11857.000000,11857.0,1.185700e+04,1.185700e+04
mean,93.0,6.0,6088.011976,2018.0,1.044110e+06,6.299014e+06
std,0.0,0.0,0.922078,0.0,9.442577e+02,9.973975e+02
min,93.0,6.0,6088.000000,2018.0,1.041556e+06,6.297580e+06
25%,93.0,6.0,6088.000000,2018.0,1.043406e+06,6.298264e+06
50%,93.0,6.0,6088.000000,2018.0,1.043953e+06,6.298733e+06
75%,93.0,6.0,6088.000000,2018.0,1.044912e+06,6.299542e+06
max,93.0,6.0,6159.000000,2018.0,1.048111e+06,6.303125e+06


In [10]:
features = list(data.columns)
features;

## Transform iris code to integer value

In [11]:
data['iris']=data['DCIRIS'].apply(lambda x: int(x[:5]+x[6:])) 

In [12]:
# drop  inconveniant iris code

data.drop(columns=['DCIRIS','REG','DEP','DEPCOM'],inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11857 entries, 0 to 11856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AN          11857 non-null  int64  
 1   TYPEQU      11857 non-null  object 
 2   LAMBERT_X   11857 non-null  float64
 3   LAMBERT_Y   11857 non-null  float64
 4   QUALITE_XY  11857 non-null  object 
 5   LAB         11857 non-null  object 
 6   CLASSE      11857 non-null  object 
 7   iris        11857 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 741.2+ KB


In [13]:
pd.unique(data['AN']) # observations only for 2008

array([2018])

In [14]:
pd.unique(data['QUALITE_XY']) # coordinates quality of X and Y

array(['Bonne', 'Mauvaise', 'Acceptable'], dtype=object)

In [15]:
data['QUALITE_XY'].value_counts()

Bonne         11598
Mauvaise        157
Acceptable      102
Name: QUALITE_XY, dtype: int64

## Geographic to cartesian transformation

The goal is to convert RGF93 Lambert 93 into WGS84 (world geographic system 1984), - to get the longitude and the latitude

Documentation sur le package

https://pypi.org/project/lambert/

In [16]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801


In [17]:
# create coordinates input list

lambert_x = list(data['LAMBERT_X'])
lamber_y = list(data['LAMBERT_Y'])
coord = zip(lambert_x,lamber_y)

In [18]:
longitude = []
latitude = []
for x, y in coord:
    pt = convertToWGS84Deg(x, y, Lambert93)
    longitude.append(pt.getX())
    latitude.append(pt.getY())

In [19]:
data['longitude']=longitude

In [20]:
data['latitude']=latitude

In [21]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,longitude,latitude
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,7.321389,43.723333
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,7.278395,43.739469
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,7.278369,43.739746
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,7.269413,43.741919
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,7.258674,43.742321


In [22]:
data.to_csv('../raw_data/data_provis.csv')

### Encode Quality_XY

In [23]:
enc = OneHotEncoder(sparse=False)

In [24]:
enc.fit(data[['QUALITE_XY']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [25]:
pd.unique(data['QUALITE_XY'])

array(['Bonne', 'Mauvaise', 'Acceptable'], dtype=object)

In [26]:
quality_xy_encoded = enc.transform(data[['QUALITE_XY']])
quality_xy_encoded

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [27]:
data["Q_XY_acceptable"],data["Q_XY_bonne"],data['Q_XY_mauvaise']=quality_xy_encoded.T

In [28]:
data['Q_XY_bonne'].value_counts()

1.0    11598
0.0      259
Name: Q_XY_bonne, dtype: int64

In [29]:
data['Q_XY_mauvaise'].value_counts()

0.0    11700
1.0      157
Name: Q_XY_mauvaise, dtype: int64

In [30]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,longitude,latitude,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,7.321389,43.723333,0.0,1.0,0.0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,7.278395,43.739469,0.0,1.0,0.0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,7.278369,43.739746,0.0,1.0,0.0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,7.269413,43.741919,0.0,1.0,0.0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,7.258674,43.742321,0.0,0.0,1.0


### Numerical features

In [31]:
features_num = ['REG',
 'DEP',
 'DEPCOM',
 'iris',
 'AN','LAMBERT_X',
 'LAMBERT_Y']

In [32]:
data['TYPEQU']

0        F106
1        D233
2        G104
3        A505
4        A505
         ... 
11852    D237
11853    D242
11854    D301
11855    E101
11856    E101
Name: TYPEQU, Length: 11857, dtype: object

In [33]:
# number of different iris => not only irises with the tramway line
len(pd.unique(data['iris']))

110

### Categorical features

In [34]:
features_cat = ['LAB','CLASSE','QUALITE_XY','TYPEQU']
features_cat

['LAB', 'CLASSE', 'QUALITE_XY', 'TYPEQU']

In [35]:
data['iris'].value_counts()

60882203    873
60882201    725
60882202    435
60882401    414
60882402    336
           ... 
60883002      1
60883301      1
60882805      1
60883102      1
60880903      1
Name: iris, Length: 110, dtype: int64

In [36]:
len(pd.unique(data['LAMBERT_X']))

4915

In [37]:
len(pd.unique(data['LAMBERT_Y']))

4866

In [38]:
data[features_cat]

,LAB,CLASSE,QUALITE_XY,TYPEQU
0,Centre équestre,Sport,Bonne,F106
1,Masseur kinésithérapeute D235,Santé,Bonne,D233
2,Information touristique,Tourisme,Bonne,G104
3,Agence immobilière,Services a,Bonne,A505
4,Agence immobilière,Services a,Mauvaise,A505
...,...,...,...,...
11852,Pédicure,Santé,Bonne,D237
11853,Diététicien,Santé,Bonne,D242
11854,Pharmacie,Santé,Bonne,D301
11855,Taxi,Transport,Bonne,E101


In [39]:
pd.unique(data['LAB'])

array(['Centre équestre', 'Masseur kinésithérapeute D235',
       'Information touristique', 'Agence immobilière',
       'Magasin de vêtements', 'Établissement santé court séjour',
       'Établissement santé moyen séjour',
       'Établissement santé long séjour', 'Psychologue', 'Pharmacie',
       'Hôtel', "Banque, Caisse d'épargne", 'Pompes funèbres',
       'Bureau de poste', 'Réparation auto et de matériel agicole',
       'Contrôle technique automobile', 'École de conduite',
       'Électricien', 'Entreprise générale du bâtiment', 'Coiffure',
       'Vétérinaire', 'Restaurant', 'Pressing', 'Institut de beauté',
       'Supermarché', 'Grande surface de bricolage', 'Supérette',
       'Épicerie', 'Boulangerie', 'Boucherie charcuterie',
       'Librairie papeterie journaux', "Magasin d'équipements du foyer",
       'Magasin de chaussures',
       "Magasin d'électroménager, audio vidéo B306",
       'Magasin de meubles',
       "Magasin d'art. de sports

In [40]:
data['LAB'].value_counts(ascending=True);

In [41]:
len(pd.unique(data['TYPEQU'])) # unique values of equipement type

152

In [42]:
data['TYPEQU'].value_counts(); # for meaning refer to 'LAB'

In [43]:
len(pd.unique(data['LAB'])) # Equipement label

152

In [44]:
len(pd.unique(data['CLASSE']))

7

In [45]:
data.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,longitude,latitude,Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,7.321389,43.723333,0.0,1.0,0.0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,7.278395,43.739469,0.0,1.0,0.0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,7.278369,43.739746,0.0,1.0,0.0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,7.269413,43.741919,0.0,1.0,0.0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,7.258674,43.742321,0.0,0.0,1.0


### Devision of LAB in categories

Produce a modification of CLASSE variable to improve grouping and compare with ML clustering

In [46]:
# Size of categories

data['CLASSE'].value_counts() # pb of imbalance

Services a    5263
Santé         3125
Commerces     2403
Transport      443
Tourisme       310
Enseigneme     200
Sport          113
Name: CLASSE, dtype: int64

In [47]:
pd.unique(data['LAB'][data['CLASSE']=='Sport']);

In [48]:
cat_sport =['Athlétisme', 'Salles de remise en forme',
       'Salles multisports (gymnase)', 'Salles spécialisées',
       'Salles de combat', 'Bassin de natation',
       'Plateaux et terrains de jeux extérieurs F112',
       'Terrain de grands jeux', 'Sports nautiques', 'Tennis', 'Roller', 'Salles non spécialisées',
       'Sports de glace', 'Boulodrome']
cat_leasure = ['Centre équestre', 'Théâtre', 'Port de plaisance', 'Musée', 'Cinéma', 'Bowling']

In [49]:
pd.unique(data['LAB'][data['CLASSE']=='Santé']);

In [50]:
cat_medical = ['Masseur kinésithérapeute D235',
       'Établissement santé court séjour',
       'Établissement santé moyen séjour',
       'Établissement santé long séjour', 'Psychologue', 'Pharmacie',
       'Centre de santé', 'Dialyse', 'Médecin généraliste',
       'Spécialiste en cardiologie',
       'Spécialiste en dermatologie vénéréologie D206',
       'Spécialiste en psychiatrie', 'Spécialiste en ophtalmologie',
       'Spécialiste en oto',
       'Spéc. en radiodiagnostic et imagerie médicale D213',
       'Spécialiste en gynécologie (médicale et/ou obstétrique)',
       'Chirurgien dentiste', 'Sage', 'Infirmier', 'Orthophoniste',
       'Orthoptiste', 'Diététicien',
       'Enfants handicapés.: soins à domicile',
       'Adultes handicapés : services d’aide',
       'Spécialiste en pédiatrie', 'Audio prothésiste',
       "Laboratoire d'analyses et de biologie médicales", 'Ambulance',
       'Ergothérapeute', 
       'Établissement psychiatrique avec hébergement D105',
       'Structures psychiatriques en ambulatoire D110', 'Crèche',
       'Spécialiste en gastro-entérologie', 'Spécialiste en pneumologie',
       'Spécialiste en stomatologie',
       "Personnes âgées : services d'aide",
       'Personnes âgées : soins à domicile', 'Psychomotricien',
       'Adultes handicapés : hébergement', 'Centre médecine préventive',
       'Centre lutte cancer', 'Urgences',
       'Enfants handicapés : hébergement', 'Travail protégé',
       'Maternité',
       'Hospitalisation à domicile',
       'Adultes handicapés : services de soins à domicile']

cat_socio_med = ['Personnes âgées : hébergement',
                 'Autres établissements pour adultes et familles en difficulté',
                 "Aide sociale à l'enfance : action éduc.",
                 "CHRS Centre d'héberg. et de réadapt. sociale",
                 'Personnes âgées : foyers restaurants',
                 "Aide sociale à l'enfance : hébergement",
                  "Centre accueil demandeur d'asile"]

In [51]:
pd.unique(data['LAB'][data['CLASSE']=='Tourisme']);

In [52]:
cat_tourist = ['Information touristique', 'Hôtel', 'Agence de voyages']

In [53]:
pd.unique(data['LAB'][data['CLASSE']=='Services a']);

In [54]:
cat_general_service = ['Bureau de poste',                   
       'Réseau partenarial pôle emploi',
       'Direction Régionale des Finances Publiques (DRFIP)',
       'Direction Départementale des Finances Publiques (DDFIP)',
       'Agence de travail temporaire', 'Réseau spécialisé Pôle emploi',
       'Réseau de proximité Pôle Emploi', 'Police',
       'Tribunal de grande instance (TGI)', "Tribunal d'instance (TI)",
       "Conseil départemental d'accès au droit (CDAD)", 'Gendarmerie',
       "Conseil des prud'hommes (CPH)", 'Relais poste',
       'Tribunal de commerce (TCO)']

In [55]:
pd.unique(data['LAB'][data['CLASSE']=='Commerces']);

In [56]:
cat_commerce = ['Supermarché', 'Supérette', 'Épicerie',
       'Boulangerie', 'Boucherie charcuterie',
       'Librairie papeterie journaux', "Magasin d'équipements du foyer",
       
       "Magasin d'électroménager, audio vidéo B306",      
       "Magasin d'art. de sports et de loisirs B308",
       'Droguerie, quincaillerie - bricolage', 
       'Magasin de matériel médical et orthopédique', 'Poissonnerie',
       'Produits surgelés',
               'Pompes funèbres',
               'Électricien', 'Vétérinaire', 'Pressing']

cat_commertce_x = ['Agence immobilière', "Banque, Caisse d'épargne", 'Coiffure', 'Restaurant',
                  'Institut de beauté', 'Pédicure',
                  'Magasin de vêtements', 'Magasin de chaussures',
                  'Parfumerie', 'Horlogerie Bijouterie', 
                   'Fleuriste', 'Magasin d’optique',]

cat_grande_surface = ['Grande surface de bricolage', 'Magasin de meubles', 
                      'Station service', 'Magasin de revêtements murs et sols', 'Hypermarché',
                     'Réparation auto et de matériel agicole', 'Contrôle technique automobile',
                     'Entreprise générale du bâtiment']

In [57]:
pd.unique(data['LAB'][data['CLASSE']=='Enseigneme']);

In [58]:
cat_high_educ = [ 'Autre formation post bac non universitaire',
                 'Formation commerce',
                 'Enseignement général supérieur privé', 'STS CPGE', 'UFR',
                 'Institut universitaire',
                 'Enseignement général supérieur privé', 'UFR',
                 'Institut universitaire',
                 'Autre enseignement supérieur', 'Résidence universitaire',
                 'Restaurant universitaire']

cat_schools = ['École élémentaire', 'Collège',
       "Lycée d'enseignement professionnel",
       "Lycée d'enseignement général et/ou techno.",
       'École maternelle',
       'Formation santé', 'Formation commerce',
       'STS CPGE', 
       "Centre de formation d'apprentis (hors agriculture)",
       'SEP : Section enseignement professionnel',
              'École de conduite',]

In [59]:
pd.unique(data['LAB'][data['CLASSE']=='Transport']);

In [60]:
cat_transport = ['Taxi', 'Gare de voyageurs d’intérêt national E108',
       'Gare de voyageurs d’intérêt local',
       'Gare de voyageurs d’intérêt régional',
                'Location auto-utilitaire légers',]

### Create columns for each category

In [61]:
def is_sport(cat):
    return cat.map(lambda x: 1 if x in  cat_sport else 0)

def is_leasure(cat):
    return cat.map(lambda x: 1 if x in  cat_leasure else 0)

def is_medical(cat):
    return cat.map(lambda x: 1 if x in  cat_medical else 0)

def is_socio_med(cat):
    return cat.map(lambda x: 1 if x in  cat_socio_med else 0)

def is_tourist(cat):
    return cat.map(lambda x: 1 if x in  cat_tourist else 0)

def is_general_service(cat):
    return cat.map(lambda x: 1 if x in  cat_general_service else 0)

def is_commerce(cat):
    return cat.map(lambda x: 1 if x in  cat_commerce else 0)

def is_commerce_spec(cat):
    return cat.map(lambda x: 1 if x in  cat_commertce_x else 0)

def is_grande_surface(cat):
    return cat.map(lambda x: 1 if x in  cat_grande_surface else 0)

def is_high_educ(cat):
    return cat.map(lambda x: 1 if x in  cat_high_educ else 0)

def is_schools(cat):
    return cat.map(lambda x: 1 if x in  cat_schools else 0)

def is_transport(cat):
    return cat.map(lambda x: 1 if x in  cat_transport else 0)

In [62]:
data_cat = data.agg({
    'LAB':[is_sport, is_leasure, is_medical, is_socio_med, 
           is_tourist,is_general_service,is_commerce,is_commerce_spec,
           is_grande_surface,is_high_educ,is_schools,is_transport]
})

In [63]:
data_cat.head()

LAB                                                                   \
  is_sport is_leasure is_medical is_socio_med is_tourist is_general_service   
0        0          1          0            0          0                  0   
1        0          0          1            0          0                  0   
2        0          0          0            0          1                  0   
3        0          0          0            0          0                  0   
4        0          0          0            0          0                  0   

                                                                          \
  is_commerce is_commerce_spec is_grande_surface is_high_educ is_schools   
0           0                0                 0            0          0   
1           0                0                 0            0          0   
2           0                0                 0            0          0   
3           0                1                 0            0          0   
4           0                1                 0            0          0   

                
  is_transport  
0            0  
1            0  
2            0  
3            0  
4            0

### Merge the data

In [64]:
df = data.merge(data_cat,how='left',left_index=True, right_index=True)

/home/mchistya/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [65]:
df.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,longitude,latitude,...,"(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)"
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,7.321389,43.723333,...,0,0,0,0,0,0,0,0,0,0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,7.278395,43.739469,...,1,0,0,0,0,0,0,0,0,0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,7.278369,43.739746,...,0,0,1,0,0,0,0,0,0,0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,7.269413,43.741919,...,0,0,0,0,0,1,0,0,0,0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,7.258674,43.742321,...,0,0,0,0,0,1,0,0,0,0


### Label iris with tramway line

In [66]:
iris_t1 = pd.read_excel('../raw_data/iris_T1.xlsx')

In [67]:
iris_t1.head(3)

,iris_id,NOM_IRIS
0,60881304,Saint-Sylvestre
1,60881503,Jeanne d'Arc-Fuon-Cauda
2,60881303,Gorbella


In [68]:
list_irist1 = list(iris_t1['iris_id'])

In [69]:
len(list_irist1)

35

In [70]:
df['iris_with_t1']=df['iris'].map(lambda x: 1 if x in list_irist1 else 0)

In [71]:
df.head()

,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE,iris,longitude,latitude,...,"(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",iris_with_t1
0,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport,60880801,7.321389,43.723333,...,0,0,0,0,0,0,0,0,0,0
1,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé,60881005,7.278395,43.739469,...,0,0,0,0,0,0,0,0,0,0
2,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme,60881005,7.278369,43.739746,...,0,1,0,0,0,0,0,0,0,0
3,2018,A505,1.043816e+06,6302952.090,Bonne,Agence immobilière,Services a,60883801,7.269413,43.741919,...,0,0,0,0,1,0,0,0,0,0
4,2018,A505,1.042950e+06,6302950.000,Mauvaise,Agence immobilière,Services a,60883801,7.258674,43.742321,...,0,0,0,0,1,0,0,0,0,0


## Dataset with important features

In [72]:
list(df.columns);

In [73]:
imp_feat = [ 'iris','iris_with_t1','AN', 'longitude', 'latitude',
            ('LAB', 'is_sport'), ('LAB', 'is_leasure'), ('LAB', 'is_medical'), ('LAB', 'is_socio_med'),
            ('LAB', 'is_tourist'), ('LAB', 'is_general_service'), ('LAB', 'is_commerce'), ('LAB', 'is_commerce_spec'),
            ('LAB', 'is_grande_surface'), ('LAB', 'is_high_educ'), ('LAB', 'is_schools'), ('LAB', 'is_transport'),
            'Q_XY_acceptable', 'Q_XY_bonne', 'Q_XY_mauvaise', 'LAB', 'CLASSE']

In [74]:
df_main = df[imp_feat]

## Main datasert

In [75]:
df_main.head()

,iris,iris_with_t1,AN,longitude,latitude,"(LAB, is_sport)","(LAB, is_leasure)","(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)",...,"(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,LAB,CLASSE
0,60880801,0,2018,7.321389,43.723333,0,1,0,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Centre équestre,Sport
1,60881005,0,2018,7.278395,43.739469,0,0,1,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Masseur kinésithérapeute D235,Santé
2,60881005,0,2018,7.278369,43.739746,0,0,0,0,1,...,0,0,0,0,0,0.0,1.0,0.0,Information touristique,Tourisme
3,60883801,0,2018,7.269413,43.741919,0,0,0,0,0,...,1,0,0,0,0,0.0,1.0,0.0,Agence immobilière,Services a
4,60883801,0,2018,7.258674,43.742321,0,0,0,0,0,...,1,0,0,0,0,0.0,0.0,1.0,Agence immobilière,Services a


In [118]:
list(pd.unique(df.iris));

### Import dataset

In [76]:
df_main.to_csv('../raw_data/bpe_t1_clean.cvs')

### Aggregation par iris de different services a la population

In [77]:
def count_sport(cat):
    return cat.map(lambda x: 1 if x in  cat_sport else 0).sum()

def count_leasure(cat):
    return cat.map(lambda x: 1 if x in  cat_leasure else 0).sum()

def count_medical(cat):
    return cat.map(lambda x: 1 if x in  cat_medical else 0).sum()

def count_socio_med(cat):
    return cat.map(lambda x: 1 if x in  cat_socio_med else 0).sum()

def count_tourist(cat):
    return cat.map(lambda x: 1 if x in  cat_tourist else 0).sum()

def count_general_service(cat):
    return cat.map(lambda x: 1 if x in  cat_general_service else 0).sum()

def count_commerce(cat):
    return cat.map(lambda x: 1 if x in  cat_commerce else 0).sum()

def count_commerce_spec(cat):
    return cat.map(lambda x: 1 if x in  cat_commertce_x else 0).sum()

def count_grande_surface(cat):
    return cat.map(lambda x: 1 if x in  cat_grande_surface else 0).sum()

def count_high_educ(cat):
    return cat.map(lambda x: 1 if x in  cat_high_educ else 0).sum()

def count_schools(cat):
    return cat.map(lambda x: 1 if x in  cat_schools else 0).sum()

def count_transport(cat):
    return cat.map(lambda x: 1 if x in  cat_transport else 0).sum()
#def count_iris_t1(cat):
#    return cat.map(lambda x: 1 if x in list_irist1 esle 0).mean()

In [78]:
df_main_agg = df_main.groupby('iris').agg({
    'LAB':[count_sport, count_leasure, count_medical, count_socio_med, 
           count_tourist,count_general_service,count_commerce,count_commerce_spec,
           count_grande_surface,count_high_educ,count_schools,count_transport]
})
df_main_agg

LAB                                              \
         count_sport count_leasure count_medical count_socio_med   
iris                                                               
60880101           0             1            27               1   
60880102           0             1            17               0   
60880103           7             0            17               1   
60880201           2             2            68               1   
60880202           1             0            21               0   
...              ...           ...           ...             ...   
60883301           0             0             0               0   
60883501           0             0             0               0   
60883701           0             0             1               1   
60883801           0             1            16               1   
61590101           1             0             0               0   

                                                             \
         count_tourist count_general_service count_commerce   
iris                                                          
60880101             7                     2             35   
60880102             2                     0             40   
60880103             3                     0             37   
60880201             4                     1             50   
60880202             8                     1             20   
...                ...                   ...            ...   
60883301             0                     0              0   
60883501             0                     0              0   
60883701             0                     0              0   
60883801             2                     0              3   
61590101             0                     0              0   

                                                                   \
         count_commerce_spec count_grande_surface count_high_educ   
iris                                                                
60880101                 190                    6               2   
60880102                 226                    2               0   
60880103                 115                    3               0   
60880201                 121                   11               0   
60880202                  82                   10               2   
...                      ...                  ...             ...   
60883301                   0                    0               0   
60883501                   0                    0               0   
60883701                   0                    0               0   
60883801                  31                    1               0   
61590101                   0                    1               0   

                                        
         count_schools count_transport  
iris                                    
60880101             4               1  
60880102             0               1  
60880103             2               2  
60880201             6               7  
60880202             3               7  
...                ...             ...  
60883301             1               0  
60883501             1               0  
60883701             2               0  
60883801             7               6  
61590101             0               0  

[110 rows x 12 columns]

In [79]:
df_main_agg.index

Int64Index([60880101, 60880102, 60880103, 60880201, 60880202, 60880203,
            60880204, 60880205, 60880206, 60880301,
            ...
            60882805, 60882902, 60883002, 60883101, 60883102, 60883301,
            60883501, 60883701, 60883801, 61590101],
           dtype='int64', name='iris', length=110)

In [80]:
#df_main_agg['iris_with_t1']=
df_main_agg['iris_with_t1']=df_main_agg.index.map(lambda x: 1 if x in list_irist1 else 0)

### Aggregated main dataset 

In [81]:
df_main_agg.head(6)

LAB                                              \
         count_sport count_leasure count_medical count_socio_med   
iris                                                               
60880101           0             1            27               1   
60880102           0             1            17               0   
60880103           7             0            17               1   
60880201           2             2            68               1   
60880202           1             0            21               0   
60880203           0             0            33               0   

                                                             \
         count_tourist count_general_service count_commerce   
iris                                                          
60880101             7                     2             35   
60880102             2                     0             40   
60880103             3                     0             37   
60880201             4                     1             50   
60880202             8                     1             20   
60880203             4                     0             20   

                                                                   \
         count_commerce_spec count_grande_surface count_high_educ   
iris                                                                
60880101                 190                    6               2   
60880102                 226                    2               0   
60880103                 115                    3               0   
60880201                 121                   11               0   
60880202                  82                   10               2   
60880203                 116                    9               1   

                                       iris_with_t1  
         count_schools count_transport               
iris                                                 
60880101             4               1            0  
60880102             0               1            1  
60880103             2               2            1  
60880201             6               7            1  
60880202             3               7            0  
60880203            11               2            0

In [221]:
df_main_agg.shape

(110, 13)

In [226]:
df_main_agg.to_csv('../raw_data/bpe_t1_agg_iris_clean.csv')

# second dataset T2

In [121]:
data2 = pd.read_csv('../raw_data/BPE_T2.csv')

In [122]:
data2.head()

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,TC_2017,cadr_2015,cadr_2016,cadr_2017,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
3,4,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,10.064,-3.189,0.0,1.035,2.498,9.619,0.796,36.487,2,0
4,5,93,6,6088,06088_0301,2018,A203,1045176.2,6299125.70,Bonne,...,6.962,13.958,-8.0,-12.207,1.366,-2.400,7.657,15.211,3,5


## Descriptive statistics

In [123]:
data2.shape

(11419, 40)

In [124]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11419 entries, 0 to 11418
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fid         11419 non-null  int64  
 1   REG         11419 non-null  int64  
 2   DEP         11419 non-null  int64  
 3   DEPCOM      11419 non-null  int64  
 4   DCIRIS      11419 non-null  object 
 5   AN          11419 non-null  int64  
 6   TYPEQU      11419 non-null  object 
 7   LAMBERT_X   11419 non-null  float64
 8   LAMBERT_Y   11419 non-null  float64
 9   QUALITE_XY  11419 non-null  object 
 10  LAB         11419 non-null  object 
 11  CLASSE      11419 non-null  object 
 12  DEPCOM_2    11419 non-null  int64  
 13  NOM_COM     11419 non-null  object 
 14  IRIS        11419 non-null  int64  
 15  iris_id     11419 non-null  int64  
 16  NOM_IRIS    11419 non-null  object 
 17  TYP_IRIS    11419 non-null  object 
 18  supkm2      11419 non-null  float64
 19  denskm2     11419 non-nul

In [148]:
data2['iris']=data2['iris_id']

In [149]:
list(data2.columns)

['fid',
 'REG',
 'DEP',
 'DEPCOM',
 'DCIRIS',
 'AN',
 'TYPEQU',
 'LAMBERT_X',
 'LAMBERT_Y',
 'QUALITE_XY',
 'LAB',
 'CLASSE',
 'DEPCOM_2',
 'NOM_COM',
 'IRIS',
 'iris_id',
 'NOM_IRIS',
 'TYP_IRIS',
 'supkm2',
 'denskm2',
 'pop_2016',
 'ds_btp',
 'Agglo',
 'revenus',
 'bpe_index',
 'evol_15_24',
 '15_24_2016',
 '15_24_2017',
 'TC_2015',
 'TC_2016',
 'TC_2017',
 'cadr_2015',
 'cadr_2016',
 'cadr_2017',
 'cdi_2015',
 'cdi_2016',
 'cdi_2017',
 'ipondi_crs',
 'CL',
 'SC',
 'iris']

In [127]:
len(pd.unique(data2['TYPEQU']))

148

In [128]:
pd.unique(data2['DEPCOM'])

array([6123, 6088])

In [129]:
pd.unique(data2['ds_btp'])

array([1057.143, 2307.692, 1087.5  , 1554.348, 1157.303,      nan,
        229.299,  663.507,  375.   ,  308.333,  846.442, 1939.394,
       1233.333, 2357.143, 1869.565, 1076.596,  790.285, 1101.796,
       4217.391, 1253.676, 1978.102, 1046.025, 2470.588, 1705.882,
       1813.008, 1500.   , 1126.582,  852.861,  675.258, 1198.732,
        871.122,  980.769,  642.738,  766.355, 1596.154, 2142.857,
       1986.667, 1450.549, 1063.725, 1718.75 , 1485.437, 1807.339,
       1833.333,   93.525, 1906.977, 1450.   , 1440.789, 1527.473,
       1029.316, 1010.695, 1921.875, 1644.068, 1360.   , 1147.541,
        932.432,  739.91 ,  585.499, 1238.806, 1336.735, 1295.   ,
       1593.333,  280.569, 1493.151, 1541.502,  866.995])

In [165]:
data2['iris_type']=data2['TYP_IRIS'].map(lambda x: 1 if x =='A' else 0)

In [166]:
pd.unique(data2['TYP_IRIS'])

array(['H', 'A'], dtype=object)

In [167]:
data2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC,iris,longitude,latitude,iris_type
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636,0


## Coordinates conversion

In [154]:
# create coordinates input list

lambert2_x = list(data2['LAMBERT_X'])
lamber2_y = list(data2['LAMBERT_Y'])
coord2 = zip(lambert2_x,lamber2_y)

In [155]:
longitude = []
latitude = []
for x, y in coord2:
    pt = convertToWGS84Deg(x, y, Lambert93)
    longitude.append(pt.getX())
    latitude.append(pt.getY())

In [157]:
data2['longitude']=longitude
data2['latitude']=latitude

In [158]:
data2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,cadr_2017,cdi_2015,cdi_2016,cdi_2017,ipondi_crs,CL,SC,iris,longitude,latitude
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,1.035,2.498,9.619,0.796,36.487,2,0,61230102,7.178602,43.658636


## Encoding

In [168]:
pd.unique(data2['QUALITE_XY'])

array(['Bonne', 'Acceptable', 'Mauvaise'], dtype=object)

In [169]:
data2['QUALITE_XY'].value_counts()

Bonne         11162
Mauvaise        161
Acceptable       96
Name: QUALITE_XY, dtype: int64

In [170]:
quality_xy_enc = enc.transform(data2[['QUALITE_XY']])


In [171]:
quality_xy_enc

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [172]:
data2["Q_XY_acceptable"], data2["Q_XY_bonne"], data2['Q_XY_mauvaise']=quality_xy_enc.T

In [173]:
data.head(3)

,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,LAB,CLASSE
0,93,6,6088,06088_0801,2018,F106,1.048111e+06,6301117.467,Bonne,Centre équestre,Sport
1,93,6,6088,06088_1005,2018,D233,1.044554e+06,6302719.450,Bonne,Masseur kinésithérapeute D235,Santé
2,93,6,6088,06088_1005,2018,G104,1.044550e+06,6302750.000,Bonne,Information touristique,Tourisme


## Categories of LAB

In [180]:
pd.unique(data2['LAB']);

In [188]:
data2_cat = data.agg({
    'LAB':[is_sport, is_leasure, is_medical, is_socio_med, 
           is_tourist,is_general_service,is_commerce,is_commerce_spec,
           is_grande_surface,is_high_educ,is_schools,is_transport]
})

In [189]:
data2_cat.isna().value_counts()

(LAB, is_sport)  (LAB, is_leasure)  (LAB, is_medical)  (LAB, is_socio_med)  (LAB, is_tourist)  (LAB, is_general_service)  (LAB, is_commerce)  (LAB, is_commerce_spec)  (LAB, is_grande_surface)  (LAB, is_high_educ)  (LAB, is_schools)  (LAB, is_transport)
False            False              False              False                False              False                      False               False                    False                     False                False              False                  11857
dtype: int64

## Merge datasets

In [193]:
df2 = data2.merge(data2_cat,how='left',left_index=True, right_index=True)
df2.head(3)

/home/mchistya/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,"(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)"
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,0,0
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,1,0,0,0,0,0,0,0,0,0
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,0,0,1,0,0,0,0,0,0,0


## Label iris with tramway line 2

In [195]:
iris_t2 = pd.read_excel('../raw_data/IrisT2.xlsx')
iris_t2['iris']=iris_t2['iris_id']

In [196]:
list_irist2 = list(iris_t2['iris_id'])

In [197]:
len(list_irist2)

69

In [198]:
df2['iris_with_t2']=df2['iris'].map(lambda x: 1 if x in list_irist2 else 0)

In [199]:
df2.head(3)

,fid,REG,DEP,DEPCOM,DCIRIS,AN,TYPEQU,LAMBERT_X,LAMBERT_Y,QUALITE_XY,...,"(LAB, is_socio_med)","(LAB, is_tourist)","(LAB, is_general_service)","(LAB, is_commerce)","(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",iris_with_t2
0,1,93,6,6123,06123_0102,2018,A405,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,0,1
1,2,93,6,6123,06123_0102,2018,A505,1036999.8,6293318.37,Bonne,...,0,0,0,0,0,0,0,0,0,1
2,3,93,6,6123,06123_0102,2018,E101,1036999.8,6293318.37,Bonne,...,0,1,0,0,0,0,0,0,0,1


## Feature selection

In [207]:
df2['iris_type']=df2['TYP_IRIS']

In [206]:
df2.columns

Index([                        'fid',                         'REG',
                               'DEP',                      'DEPCOM',
                            'DCIRIS',                          'AN',
                            'TYPEQU',                   'LAMBERT_X',
                         'LAMBERT_Y',                  'QUALITE_XY',
                               'LAB',                      'CLASSE',
                          'DEPCOM_2',                     'NOM_COM',
                              'IRIS',                     'iris_id',
                          'NOM_IRIS',                    'TYP_IRIS',
                            'supkm2',                     'denskm2',
                          'pop_2016',                      'ds_btp',
                             'Agglo',                     'revenus',
                         'bpe_index',                  'evol_15_24',
                        '15_24_2016',                  '15_24_2017',
                           'TC_201

In [203]:
imp_feat2 = [ 'iris','iris_type','iris_with_t2','AN', 'longitude', 'latitude',
            ('LAB', 'is_sport'), ('LAB', 'is_leasure'), ('LAB', 'is_medical'), ('LAB', 'is_socio_med'),
            ('LAB', 'is_tourist'), ('LAB', 'is_general_service'), ('LAB', 'is_commerce'), ('LAB', 'is_commerce_spec'),
            ('LAB', 'is_grande_surface'), ('LAB', 'is_high_educ'), ('LAB', 'is_schools'), ('LAB', 'is_transport'),
            'Q_XY_acceptable', 'Q_XY_bonne', 'Q_XY_mauvaise', 'LAB', 'CLASSE']

In [213]:
df_main2=df2[imp_feat2]

In [214]:
df_main2.head(3)

,iris,iris_with_t2,AN,longitude,latitude,"(LAB, is_sport)","(LAB, is_leasure)","(LAB, is_medical)","(LAB, is_socio_med)","(LAB, is_tourist)",...,"(LAB, is_commerce_spec)","(LAB, is_grande_surface)","(LAB, is_high_educ)","(LAB, is_schools)","(LAB, is_transport)",Q_XY_acceptable,Q_XY_bonne,Q_XY_mauvaise,LAB,CLASSE
0,61230102,1,2018,7.178602,43.658636,0,1,0,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Électricien,Services a
1,61230102,1,2018,7.178602,43.658636,0,0,1,0,0,...,0,0,0,0,0,0.0,1.0,0.0,Agence immobilière,Services a
2,61230102,1,2018,7.178602,43.658636,0,0,0,0,1,...,0,0,0,0,0,0.0,1.0,0.0,Taxi,Transport


In [225]:
df_main2.to_csv('../raw_data/bpe_t2_clean.csv')

## Aggregation at iris level

In [216]:
df_main_agg2 = df_main2.groupby('iris').agg({
    'LAB':[count_sport, count_leasure, count_medical, count_socio_med, 
           count_tourist,count_general_service,count_commerce,count_commerce_spec,
           count_grande_surface,count_high_educ,count_schools,count_transport]
})
df_main_agg2

LAB                                              \
         count_sport count_leasure count_medical count_socio_med   
iris                                                               
60880101           2             1            27               2   
60880102           0             0            17               0   
60880103           5             1            17               1   
60880201           2             1            63               1   
60880202           2             1            23               0   
...              ...           ...           ...             ...   
60883102           9             0            15               1   
61230101           0             0            13               0   
61230102           3             0             9               0   
61230103           2             0            38               0   
61230107          10             0            42               0   

                                                             \
         count_tourist count_general_service count_commerce   
iris                                                          
60880101             7                     2             35   
60880102             2                     0             38   
60880103             3                     0             39   
60880201             4                     1             47   
60880202             8                     1             22   
...                ...                   ...            ...   
60883102             5                     1             16   
61230101             1                     5             16   
61230102            11                     1             20   
61230103             3                     5             17   
61230107             3                     3             41   

                                                                   \
         count_commerce_spec count_grande_surface count_high_educ   
iris                                                                
60880101                 193                    5               2   
60880102                 210                    1               0   
60880103                 126                    4               0   
60880201                 106                   10               0   
60880202                  93                   10               2   
...                      ...                  ...             ...   
60883102                  35                   21               0   
61230101                 144                   15               0   
61230102                  62                   26               0   
61230103                  68                    5               0   
61230107                  72                   13               0   

                                        
         count_schools count_transport  
iris                                    
60880101             6               1  
60880102             0               1  
60880103             2               2  
60880201             3               7  
60880202             4               9  
...                ...             ...  
60883102             9              13  
61230101             0               2  
61230102             0              12  
61230103             5               4  
61230107             8               4  

[69 rows x 12 columns]

In [218]:
len(df_main_agg2.index)

69

In [219]:
# Identify if T2 passes through the iris
df_main_agg2['iris_with_t2']=df_main_agg2.index.map(lambda x: 1 if x in list_irist2 else 0)

In [220]:
df_main_agg2.head(3)

LAB                                              \
         count_sport count_leasure count_medical count_socio_med   
iris                                                               
60880101           2             1            27               2   
60880102           0             0            17               0   
60880103           5             1            17               1   

                                                             \
         count_tourist count_general_service count_commerce   
iris                                                          
60880101             7                     2             35   
60880102             2                     0             38   
60880103             3                     0             39   

                                                                   \
         count_commerce_spec count_grande_surface count_high_educ   
iris                                                                
60880101                 193                    5               2   
60880102                 210                    1               0   
60880103                 126                    4               0   

                                       iris_with_t2  
         count_schools count_transport               
iris                                                 
60880101             6               1            1  
60880102             0               1            1  
60880103             2               2            1

In [222]:
df_main_agg2.shape

(69, 13)

In [224]:
df_main_agg2.to_csv('../raw_data/bpe_t2_agg_iris_clean.csv')